In [ ]:
import random
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import numpy as np
import math

warnings.filterwarnings('ignore')

In [ ]:
# queries = ['10b.csv', '10c.csv', '11b.csv', '11c.csv', '11d.csv', '12b.csv', '12c.csv', '13b.csv',
#                          '13c.csv', '13d.csv', '14b.csv', '14c.csv', '15b.csv', '15c.csv', '15d.csv', '16b.csv',
#                          '16c.csv', '16d.csv', '17b.csv', '17c.csv', '17d.csv', '17e.csv', '17f.csv', '18b.csv',
#                          '18c.csv', '19b.csv', '19c.csv', '19d.csv', '1b.csv', '1c.csv', '1d.csv', '20b.csv', '20c.csv',
#                          '21b.csv', '21c.csv', '22b.csv', '22c.csv', '22d.csv', '23b.csv', '23c.csv', '24b.csv',
#                          '25b.csv', '25c.csv', '26b.csv', '26c.csv', '27b.csv', '27c.csv', '28b.csv', '28c.csv',
#                          '29b.csv', '29c.csv', '2b.csv', '2c.csv', '2d.csv', '30b.csv', '30c.csv', '31b.csv', '31c.csv',
#                          '32b.csv', '33b.csv', '33c.csv', '3b.csv', '3c.csv', '4b.csv', '4c.csv', '5b.csv', '5c.csv',
#                          '6b.csv', '6c.csv', '6d.csv', '6e.csv', '6f.csv', '7b.csv', '7c.csv', '8b.csv', '8c.csv',
#                          '8d.csv', '9b.csv', '9c.csv', '9d.csv']
# DATA_PATH = "" # 80 in total
# normalizaton_imdb = 10

queries = ["{}{}.csv".format(i,x)for i in [5,7,8,10,12,13,14] for x in ["a","b","c","d","e","f","g","h","i","j"]]

print(len(queries)) # 70 in total
DATA_PATH = ""
print(DATA_PATH) 
normalization_tpch=75
normalization = normalization_tpch

#normalization = normalizaton_imdb

In [ ]:
len(queries)

In [ ]:
def conformal_score_difference(cost, latency):
  return abs(cost - latency)

def get_conformal_score(conformal_score_lambda, set_name):
  all_data = pd.DataFrame()
  for query in set_name:
      csv_file = DATA_PATH+query
      query_id = query.split('.')[0]
      try:
          df = pd.read_csv(csv_file)
          df['ConformalScore'] = conformal_score_lambda(df['Cost']/normalization, df['Actual Total Time'])
          df['QueryID'] = query_id
          df = df[~df['Operation'].str.contains('Scan')]
          df = df.iloc[[0]]
          all_data = pd.concat([all_data, df], ignore_index=True)

      except FileNotFoundError:
          print(f"File {csv_file} not found.")
      except Exception as e:
          print(f"An error occurred while processing {csv_file}: {e}")
  return list(all_data["ConformalScore"])

In [ ]:
import math
def update_get_quantile(alpha, sortedCP):
    n = len(sortedCP)
    q_hat_index = math.ceil(((n+1)*(1-alpha)))
    return sortedCP[q_hat_index-1]

In [ ]:
def quantile_stats(quantList):
  # Example list of numbers
  numbers = quantList

  # Convert the list to a NumPy array
  data = np.array(numbers)

  # Calculate mean and standard deviation
  mean = np.mean(data)
  std_dev = np.std(data)

  # Plot the data
  plt.figure(figsize=(10, 6))

  # Plot the numbers
  plt.plot(data, marker='o', label='Data')

  # Plot the mean line
  plt.axhline(mean, color='red', linestyle='--', label=f'Mean: {mean:.2f}')

  # Plot lines for 1 standard deviation from the mean
  plt.axhline(mean + std_dev, color='green', linestyle='--', label=f'Mean + 1 Std: {mean + std_dev:.2f}')
  plt.axhline(mean - std_dev, color='green', linestyle='--', label=f'Mean - 1 Std: {mean - std_dev:.2f}')

  # Labels and title
  plt.title('Deviation Plot')
  plt.xlabel('Index')
  plt.ylabel('Value')
  plt.legend()

  # Show the plot
  plt.grid(True)
  plt.show()


In [ ]:
def offline_verfication(iterations, calibration_set_size, alpha = 0.1):
  valid_rates = []
  all_quant_vals = []
  count_above_90 = 0
  for i in range(iterations):
    calibration_set = random.sample(queries, calibration_set_size)
    test_set = [elem for elem in queries if elem not in calibration_set]

    CPScores_cali = get_conformal_score(conformal_score_difference, calibration_set)
    sortedCP_cali = sorted(CPScores_cali)
    quantVal = update_get_quantile(alpha, sortedCP_cali)
    all_quant_vals.append(quantVal)

    CPScores_test = get_conformal_score(conformal_score_difference, test_set)
    num_in = 0
    for val in CPScores_test:
      if val <= quantVal:
        num_in += 1

    valid_rate = num_in / len(CPScores_test)
    if valid_rate > (1-alpha):
      count_above_90 += 1

    print("==> Iter {}, count_above/total: {}/{}, quantVal: {}, valid_rate: {:.2f}%".format(i, num_in, len(CPScores_test), quantVal, valid_rate*100))

    valid_rates.append(valid_rate)

  assert(len(valid_rates) == iterations)
  avg_valid_rate = sum(valid_rates) / len(valid_rates)
  print("\nAverage valid_rate: {:.2f}%".format(avg_valid_rate * 100))
  print(f"Number of valid_rate > {(1-alpha)*100}%: {format(count_above_90)}")

  plt.hist(valid_rates, bins=30, range=(0.5, 1.0), edgecolor='black')
  plt.title("Histogram of Valid Rate (Lastest online)")
  plt.xlabel("Valid Rate")
  plt.ylabel("Frequency")
  plt.grid(True)
  plt.show()
  quantile_stats(all_quant_vals)

In [ ]:
offline_verfication(1000, 35, 0.1)